In [6]:
import pandas as pd
import numpy as np

In [15]:
TRAIN_FILE = "./data/train.csv"
TEST_FILE = "./data/test.csv"

df_train = pd.read_csv(TRAIN_FILE)
df_test = pd.read_csv(TEST_FILE)

X, y = df_train.drop("is_converted", axis=1), df_train["is_converted"]
X_test = df_test.drop(["is_converted", "id"], axis=1)

In [16]:
def preprocess_product_dict(df: pd.DataFrame) -> dict:
    product_category_remap = {
        # IS: Integrated Solutions Inquiry - INTEGRATED SOLUTIONS
        "IS" : [
            "monitor signage",
            "commercial tv",
            "monitor",
            "monitor tv",
            "pc",
            "projector",
            "robot",
            "system ac",
            "ems",
            "rac",
            "tv",
            "regrigerator",
            "washing machine",
            "aircare",
            "vaccum cleaner",
            "styler",
            "dryer",
            "built-in/cooking",
            "home beauty",
            "water care",
            "audio/video",
            "lg thinq home",
            "smart home",
            "lg one:quick_flex",    # healthcare solutions
            "one:quick",    # healthcare solutions
            "quick",    # healthcare solutions
        ],
        # CD: B2B Product Inquiry - Commercial Display
        "CD": [
            "^(?=.*\d)[A-Za-z0-9.\s-]+$",    # tv panel model name filter
            "medical display",
            "led bloc",
            "uhd signage",
            "commercial display",
            "oled signage",
            "led signage",
            "video wall signage",
            "interactive signage",
            "high brightness signage",
            "special signage",
            "standard signage",
            "hotel tv",
            "hospital tv",
            "accessories",
            "software solution",
            "signage care solution",
            "webos",
            "pro:centric",
            "one:quick series",
            "signage",
            "display",
            'led',
            'wall',    # video wall,
            'MAGNIT',
            "centric"
        ],
        # IT: B2B Product Inquiry - IT PRODUCTS
        "IT": [
            "monitor",
            "laptop",
            "projector",
            "cloud device",
        ],
        # HE: B2B Product Inquiry - HVAC/ESS
        "HE": [
            "control",
            "ventilation",
            "vrf",
            "split",
            "multi-split",
            "single-split",
            "chiller",
            "heating",
            "energy storage system",
            "ess",
            "solar",
            "heat",
            "water",
            "air condition",
            "air"
        ],
        # CM: B2B Product Inquiry - Compressor and Motor
        "CM": [
            "reciprocating compressor",
            "rotary compressor",
            "scroll compressor",
            "compressor",
            "motor",
        ],
        # RB: B2B Product Inquiry - Robot
        "RB": [
            "LG CLOi UV-C Bot",
            "LG CLOi ServeBot",
            "Shelf type",
            "Drawer type",
            "LG CLOi GuideBot",
            "CLOi",
            "UV-C",
            "Bot",
        ],
        # AM: B2B Product Inquiry - ADVANCED MATERIALS
        "AM": [
            "Antimicrobial",
            "Porcelain enamel",
            "Porcelain",
            "enamel",
            "Specialty glass",
            "Specialty",
            "glass",
        ],
        # others
        "others": [
            "[\(\)]",
            "MISSING_VALUE",    # missing value
            "commercial laundry"    # B2B Product Inquiry
            "others",
            "ohters",
            "other",
            "ohter",
            'otros',
            'outros',
            'error',
            "etc",
            *(
                df["product_category"]
                .value_counts()
                .loc[df["product_category"].value_counts() < 500]
                .index.tolist()
            )
        ]
    }
    
    return product_category_remap

In [17]:
def preprocess_product_category(df: pd.DataFrame) -> pd.DataFrame:
    # making dictionary depending on LG official website
    product_category_remap = preprocess_product_dict(df)
    
    # filling missing value: "0"
    df_pc = df['product_category'].fillna("MISSING_VALUE", inplace=False)

    # preprocessing by category
    for key, value in product_category_remap.items():
        regex = '|'.join(value)
        
        df_pc.loc[df_pc.str.contains(
            regex,         # chekcing string pattern
            na=False,     # missing value return is boolean
            case=False    # ignore upper and lower
        )] = key
        
    # translating foreign language
    ''' code '''
    
    return df_pc

df_pc = preprocess_product_category(df_train)
X['product_category'] = df_pc

df_pc = preprocess_product_category(df_test)
X_test['product_category'] = df_pc

C:\Users\kjmin\AppData\Local\Temp\ipykernel_8992\2603859965.py:12: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_pc.loc[df_pc.str.contains(


In [20]:
X['product_category'].unique()

array(['HE', 'others', 'IS', 'CD', 'IT'], dtype=object)